In [9]:
import functools
import numpy as np
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import os
import pandas as pd

from tensorflow import keras as keras
from keras.utils import to_categorical


df =  pd.read_csv('Documents/DAFeatures_Train.csv')
print(df.shape)
df.describe()

#create a dataframe with all training data except the target column
X_train = df.drop(columns=['File Type'])

#check that the target variable has been removed
print(X_train.head())

Using TensorFlow backend.


(7676, 9)
   Files Deleted  Files Created  Files Written  Directories Created  \
0              0              0              0                    0   
1              0              6              4                    8   
2              8              6              7                   15   
3              0              0              0                    1   
4              5             71             70                   11   

   Opened REG keys  DLLs Loaded  Hosts  VirusTotal  
0                0            3      0           0  
1              277           19      4           0  
2              309           74      0           0  
3              185           15      0           0  
4              524           49      0           0  


In [10]:
#create a dataframe with only the target column
y_train = df[['File Type']]

#view dataframe
print(y_train.head())


   File Type
0          6
1          1
2          6
3          6
4          6


In [11]:
#one-hot encode target column
y_train = to_categorical(df['File Type'])

#vcheck that target column has been converted
print(y_train[0:10])

[[0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]
print (n_cols)

#add model layers
model.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(7, activation='softmax'))
#model.add(Dense(7))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#train model
model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

8
Train on 6140 samples, validate on 1536 samples
Epoch 1/30
6140/6140 [==============================] - 5s 839us/step - loss: 1.0632 - accuracy: 0.7883 - val_loss: 0.8552 - val_accuracy: 0.8737
Epoch 2/30
6140/6140 [==============================] - 2s 299us/step - loss: 0.6092 - accuracy: 0.8749 - val_loss: 0.5981 - val_accuracy: 0.8822
Epoch 3/30
6140/6140 [==============================] - 2s 339us/step - loss: 0.4190 - accuracy: 0.8883 - val_loss: 0.3159 - val_accuracy: 0.9069
Epoch 4/30
6140/6140 [==============================] - 2s 343us/step - loss: 0.3266 - accuracy: 0.9029 - val_loss: 0.2899 - val_accuracy: 0.9128
Epoch 5/30
6140/6140 [==============================] - 2s 345us/step - loss: 0.3156 - accuracy: 0.9098 - val_loss: 0.2588 - val_accuracy: 0.9095
Epoch 6/30
6140/6140 [==============================] - 2s 392us/step - loss: 0.2871 - accuracy: 0.9129 - val_loss: 0.2781 - val_accuracy: 0.9232
Epoch 7/30
6140/6140 [==============================] - 2s 358us/step - lo

In [18]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   

model.save('MalDA.model')

 
#pred_test= model.predict(X_test)
#scores2 = model.evaluate(X_test, y_test, verbose=0)
#print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

Accuracy on training data: 0.9314747452735901% 
 Error on training data: 0.06852525472640991


In [20]:
#example on how to use our newly trained model on how to make predictions on unseen data (we will pretend our new data is saved in a dataframe called 'test_X').
test_file =  pd.read_csv('Documents/DAFeatures_Test.csv')
X_test = test_file.drop(columns=['File Type'])

#one-hot encode target column
y_test = to_categorical(test_file['File Type'])

#vcheck that target column has been converted
print(y_test[0:30])

print(X_test.shape)
pred_test= model.predict_classes(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))  

print(pred_test[0:30])



[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]
(2562, 8)
Accuracy on test data: 0.931693971157074% 
 Error on test data: 0.06830602884292603
[6 6 4 6 4 3 6 6 3 1 2 6 6 4 4 5 6 5 6 4 6 6 6 6 5 6 6 0 6 6]
